# Set Up NVFLARE in POC Mode

[POC mode](https://nvflare.readthedocs.io/en/main/user_guide/poc_command.html) allows users to test the features of a full FLARE deployment on a single machine, without the overhead of a true distributed deployment.

Compared to the FL Simulator, where the job run is automated on a single system, POC mode allows you to establish and connect distinct server and client "systems" which can then be orchestrated using the FLARE Console.  This can be useful in preparation for a distributed deployment.

>It is ideal to start your NVFLARE system in POC mode from a **terminal**, not from a notebook. The terminal's virual env. must match the kernel's virtual env. In our case, we are using 'nvflare_example'.

To get started, let's look at the NVFlare CLI usage for the ``poc`` subcommand:

In [ ]:

! nvflare poc -h


### Preparing the POC environment
Before running POC mode, there are a couple important environment variables that should be set.

First, to simplify deploying the example apps in the NVFlare GitHub repo, you can set `NVFLARE_HOME` to the root of the GitHub clone.  

In [ ]:
import os
NVFLARE_HOME=os.path.abspath(os.path.join(os.getcwd(), "../.."))
%env NVFLARE_HOME={NVFLARE_HOME}
! echo $NVFLARE_HOME

By default, POC mode uses a temporary workspace in /tmp/nvflare/poc.

If you prefer to set a workspace in different location than the default, you can set the environment variable NVFLARE_POC_WORKSPACE 

```
%env NVFLARE_POC_WORKSPACE=/your/new/location/path

```

In [ ]:
%env  NVFLARE_POC_WORKSPACE=/tmp/nvflare/poc
!echo $NVFLARE_POC_WORKSPACE 

If you want to use the default poc workspace, ``/tmp/nvflare/poc``, you don't need set ``NVFLARE_POC_WORKSPACE``. 

### Preparing the POC workspace

Now that we've configured out POC environment, we can prepare the POC workspace.  By default, the following command will generate POC packages for a server and two clients.
nvflare poc --prepare
```
    nvflare poc --prepare 
```

You can specify a different number of clients with the -n option:

```
   nvflare poc --prepare -n N
```

If you running the POC prepare command, there is a prompt for user to answer.

### Start NVFlare system in POC mode

When starting the POC deployment, it's necessary to use a separate terminal since the `nvflare poc --start` command will run  in the foreground emitting output from the server and any connected clients.

Also note that `nvflare poc --start` starts all participants, including the admin console. We want to start the 2 clients and 1 server, but without the FLARE console (aka Admin Console), as we are going to interact with the system from a notebook.

Create a terminal to execute the following command (in JupyterLab or host terminal), then

```
    nvflare poc --start -ex admin
```
**Note:**
    Using ```%%bash -bg``` to run the above command in a code cell may not always work

### Check system information

In [ ]:

import os
from nvflare.fuel.flare_api.flare_api import new_secure_session

poc_workspace = os.getenv('NVFLARE_POC_WORKSPACE')

workspace = "/tmp/nvflare/poc" if  poc_workspace is None else poc_workspace

default_poc_prepared_dir = os.path.join(workspace, "example_project/prod_00")
admin_dir = os.path.join(default_poc_prepared_dir, "admin@nvidia.com")
sess = new_secure_session("admin@nvidia.com", admin_dir)
sys_info = sess.get_system_info()

print(f"Server info:\n{sys_info.server_info}")
print("\nClient info")
for client in sys_info.client_info:
    print(client)

Once the system is up, we are now ready to go back to the notebook to work on your project.

### Stop FLARE system in POC

Once you are done with the FLARE system, you can shut down the poc system.
From terminal, issue the following command stop poc in the terminal
```
   nvflare poc --stop
```


### Clean up POC workspace

If you want to clean up the POC workspace and delete the poc workspace directory containing all packages you may have prepared, you can use the following command:

In [ ]:
! nvflare poc --clean

You can check if the nvflare system is shutdown cleanly with:

```
ps -eaf | grep nvflare
```

If you see output like the following, then nvflare systems are still running:

```
510535    1932  1 18:54 pts/1    00:00:03 python3 -u -m nvflare.private.fed.app.client.client_train -m /tmp/workspace/example_project/prod_00/site-1/startup/.. -s fed_client.json --set secure_train=true uid=site-1 org=nvidia config_folder=config
510539    1932  1 18:54 pts/1    00:00:03 python3 -u -m nvflare.private.fed.app.client.client_train -m /tmp/workspace/example_project/prod_00/site-2/startup/.. -s fed_client.json --set secure_train=true uid=site-2 org=nvidia config_folder=config
510543    1932  1 18:54 pts/1    00:00:04 python3 -u -m nvflare.private.fed.app.server.server_train -m /tmp/workspace/example_project/prod_00/localhost/startup/.. -s fed_server.json --set secure_train=true org=nvidia config_folder=config
```

If you have already used ``nvflare poc --clean`` to delete the workspace, you may need to kill the processes manually.
